<a href="https://colab.research.google.com/github/AdrianDiazGaxiola/Proyecto-Final/blob/main/Precipitaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se importan las librerías a utilizar

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

Se leen los archivo con los datos de registros de Precipitación en México y se guardan en un diccionario. También se lee el archivo con los datos de las temperaturas a nivel nacional

In [11]:
Prec = {}

for i in range(1985, 2021):
  url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/blob/main/Data/Precipitacion/%dPrecip.xls?raw=true" % i
  if (i==2020):
    url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/blob/main/Data/Precipitacion/2020Precip.xlsx?raw=true"
  Prec[i] = pd.read_excel(url, header=1)

url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/raw/main/Data/Temperaturas%20Completo%20por%20Entidad.csv"
temperaturas = pd.read_csv(url) 

Se crea un dataframe por estado para guardar los datos de las precipitaciones de cada uno de estos

In [12]:
#Se guardan las abreviaturas de los estados en una lista
estados_abv = ['AGU', 'BCN', 'BCS', 'CAM', 'CHP', 'CHH', 'CMX', 'COA', 'COL', 'DUR', 'GUA', 'GRO', 'HID', 'JAL', 'MEX', 'MIC', 'MOR', 'NAY', 'NLE', 'OAX', 'PUE', 'QUE', 'ROO', 'SLP', 'SIN', 'SON', 'TAB', 'TAM', 'TLA', 'VER', 'YUC', 'ZAC', 'NAC']

#Se guardan en una lista los nombres de los estados tal y como vienen en el archivo inicial.
df = pd.DataFrame(Prec[1985])
estados_comp = df['ENTIDAD'].tolist()

anios = range(1985, 2021) #Lista con los años de los dataframe
prec_por_entidad = {}

#Ciclo para guardar los datos de cada estado.
for i in range(33):
  df_prec_entidad = pd.DataFrame(columns=['ENTIDAD', 'ENE', 'FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEP', 'OCT', 'NOV', 'DIC', 'ANUAL']) #Dataframe en el que se guardará los datos de un solo estado.
  
  for j in range (1985, 2021):
    l = j - 1985 #Variable para guardar el año
   
    df = pd.DataFrame (Prec[j])    
   
    df_prec_entidad = df_prec_entidad.append(df.loc[i], ignore_index=True) #Se copia la fila con los datos del estado correspondiente al nuevo dataframe
   
    df_prec_entidad.at[l, 'ENTIDAD'] = anios[l] #Se guarda el número del año correspondiente en el dataframe

  df_prec_entidad.rename(columns = {'ENTIDAD':'ANIO'}, inplace = True)

  prec_por_entidad[estados_abv[i]] = df_prec_entidad #Se guarda en el diccionario de las precipitaciones por entidad el dataframe creado en el ciclo, identificado por la abreviatura de la entidad


Ejemplos de los dataframes por estado creados

In [ ]:
print (prec_por_entidad['SIN'].head())
print (prec_por_entidad['NAC'].head())

Se crea un dataframe con todos los datos de precipitación

In [14]:
df_completo = pd.DataFrame(columns=["ANIO", "MES", "ENTIDAD", "Precipitacion Media"])

#Se guardan las abreviaturas de los estados en una lista
estados_abv = ['AGU', 'BCN', 'BCS', 'CAM', 'CHP', 'CHH', 'CMX', 'COA', 'COL', 'DUR', 'GUA', 'GRO', 'HID', 'JAL', 'MEX', 'MIC', 'MOR', 'NAY', 'NLE', 'OAX', 'PUE', 'QUE', 'ROO', 'SLP', 'SIN', 'SON', 'TAB', 'TAM', 'TLA', 'VER', 'YUC', 'ZAC', 'NAC']

#Se guardan las abreviaturas de los meses en una lista
df = Prec[1985]
meses_abv = df.columns.values.tolist()
meses_abv.remove("ENTIDAD")

for i in range(1985, 2021):
  df = Prec[i]

  for j in meses_abv:
    m = 0 #Variable para acceder a las filas de los dataframes (una por entidad)
    for k in estados_abv:
 
      #Se copia la precipitación media correspondiente a la entidad, mes y año, donde j es el mes y m la fila de la entidad 
      lluvia = df.at[m, j]

      #Se adjuntan al dataframe con los datos completos el año, mes, entidad y precipitación media correspondientes
      df_completo = df_completo.append({"ANIO": i, "MES": j, "ENTIDAD": k, "Precipitacion Media": lluvia}, ignore_index=True)

      m += 1
     
print (df_completo)

       ANIO    MES ENTIDAD  Precipitacion Media
0      1985    ENE     AGU             4.914729
1      1985    ENE     BCN            12.150562
2      1985    ENE     BCS            30.304723
3      1985    ENE     CAM            20.943153
4      1985    ENE     CHP            40.714546
...     ...    ...     ...                  ...
15439  2020  ANUAL     TLA           538.400000
15440  2020  ANUAL     VER          1548.600000
15441  2020  ANUAL     YUC          1802.200000
15442  2020  ANUAL     ZAC           406.300000
15443  2020  ANUAL     NAC           722.500000

[15444 rows x 4 columns]


Se combinan en un dataframe los datos de temperaturas con los datos de precipitación

In [ ]:
#Como los registros de precipitación son anuales, se utilizarán los registros de temperaturas anuales para futuro análisis.
df = temperaturas.loc[temperaturas['MES']=='ANUAL']

#Se guardan en un dataframe los datos de temperaturas de los años de los cuales se tienen registros se tienen registros de precipitación.
df1 = df.loc[df['ANIO']==2000]
df2 = df.loc[df['ANIO']>=2005]
df = df1.append(df2)
df = df.reset_index(drop=True)

#Se guardan en una lista los años de los cuales se tienen registros de precipitación en México.
Anios = list(range(2000, 2021))
del Anios[1:5]

#Se guardan las abreviaturas de los estados en una lista
estados_abv = ['AGU', 'BCN', 'BCS', 'CAM', 'CHP', 'CHH', 'CMX', 'COA', 'COL', 'DUR', 'GUA', 'GRO', 'HID', 'JAL', 'MEX', 'MIC', 'MOR', 'NAY', 'NLE', 'OAX', 'PUE', 'QUE', 'ROO', 'SLP', 'SIN', 'SON', 'TAB', 'TAM', 'TLA', 'VER', 'YUC', 'ZAC', 'NAC']

#Se crea una lista para guardar en ella los datos de precipitacion para después adjuntarla al dataframe
lluvias = []

for i in Anios:
  l = 0
  for j in estados_abv:
    lluvia = precipitacion.at[l, str(i)] #Se copia el valor del registro de precipitación del estado y año correspondiente
    lluvias.append(lluvia)
    l += 1

df['Precipitacion'] = lluvias

print(df.head())
print(precipitacion.head())
print(df.tail())
print(precipitacion.tail())